In [1]:
import pandas as pd
import numpy as np
import datetime
import os
# Add the new folders of Jian later

In [2]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            
last_saturday=datetime.datetime.now().date()-datetime.timedelta(days=datetime.datetime.now().date().weekday()+2) #+7

write_folder="/home/jian/celery/Append_to_celery_LiveRamp/monthly_update/output_"+str(last_saturday)+"/"
try:
    os.stat(write_folder)
except:
    os.mkdir(write_folder)

week_beginning=last_saturday-datetime.timedelta(days=14) # To be changed to 14

recent_2_weeks=[last_saturday-datetime.timedelta(days=x*7) for x in range(2)]
recent_2_weeks

[datetime.date(2019, 8, 10), datetime.date(2019, 8, 3)]

In [3]:
# Checking for the recent weeks that haven't moved to the folder
possible_recent_folders=["/home/jian/BigLots/MediaStorm_"+str(x)+"/" for x in recent_2_weeks]
recent_file_list=[]
for dirc in possible_recent_folders:
    list_recent=[x for x in list(recursive_file_gen(dirc)) if ("Master" in x) & (".txt" in x) ]
    recent_file_list=recent_file_list+list_recent
recent_file_df=pd.DataFrame({"path":recent_file_list,"date":[datetime.datetime.strptime(x.split("MasterWeekly")[1][:8],"%Y%m%d").date()-datetime.timedelta(days=3) for x in recent_file_list]},index=[x for x in range(len(recent_file_list))])


list_1_after_201806=[x for x in list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/")) if ("Master" in x) & (".txt" in x) ]
folder_date=[datetime.datetime.strptime(x.split("/")[len(x.split("/"))-2].split("_")[1],"%Y-%m-%d").date() for x in list_1_after_201806]
df_1_after_201806=pd.DataFrame({"date":folder_date,"path":list_1_after_201806},index=[x for x in range(len(list_1_after_201806))])
df_1_after_201806['date'].apply(lambda x: x.weekday()).unique()
df_1_after_201806=df_1_after_201806.sort_values("date").reset_index()
df_1_after_201806=df_1_after_201806[df_1_after_201806['date']>week_beginning]
del df_1_after_201806['index']

new_2_biweekly_files=df_1_after_201806.append(recent_file_df)

print(len(new_2_biweekly_files))
new_2_biweekly_files=new_2_biweekly_files['path'].unique().tolist()

2


In [4]:
new_2_biweekly_files

['/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-08-03/MediaStormMasterWeekly20190806-114102-298.txt',
 '/home/jian/BigLots/MediaStorm_2019-08-10/MediaStormMasterWeekly20190813-120648-031.txt']

In [5]:
if len(new_2_biweekly_files)==2: # Changed to weekly
    df_all_new_master = pd.DataFrame()
    for file in new_2_biweekly_files:
        df = pd.read_csv(file,nrows = None,sep= '|',usecols = ['customer_id_hashed','email_address_hash','customer_zip_code'],
                         dtype =str)
        df_all_new_master = df_all_new_master.append(df,ignore_index = True)
    print(len(df_all_new_master.index))
    df_all_new_master = df_all_new_master.drop_duplicates('email_address_hash')
    print(len(df_all_new_master.index))

231295
231295


In [6]:
# Remove previous week ids

previous_id_files_lists=list(recursive_file_gen("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/"))

# previous_id_files_lists=[x for x in previous_id_files_lists if "Copy of 48 Stores_727.csv" not in x]
previous_id_files_lists=[x for x in previous_id_files_lists if ".csv" in x]
print(len(previous_id_files_lists))

72


In [7]:
previous_id_files_lists

['/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/T_Quadrant I_P_L_2019Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/T_Quadrant I_S_H_2019Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/T_Quadrant I_S_L_2019Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/T_Quadrant I_T_H_2019Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/T_Quadrant I_T_M_2019Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/C_Quadrant III_P_H_2019Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/C_Quadrant III_P_M_2019Q1.csv',
 '/home/jian/Projects/Big_Lots

In [8]:
import glob
uploaded_new_seg=list(recursive_file_gen("/home/jian/celery/Append_to_celery_LiveRamp/monthly_update/"))
uploaded_new_seg=[x for x in uploaded_new_seg if ".csv" in x]

previous_id_files_lists=previous_id_files_lists+uploaded_new_seg

In [9]:
for file_uploaded in previous_id_files_lists:
    df = pd.read_csv(file_uploaded,usecols = ['email_address_hash'])
    previous_email_set=set(df['email_address_hash'].tolist())
    df_all_new_master = df_all_new_master[~df_all_new_master['email_address_hash'].isin(previous_email_set)]
    print(datetime.datetime.now(),len(df_all_new_master.index),file_uploaded)

2019-08-14 10:15:53.164073 231293 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/T_Quadrant I_P_L_2019Q1.csv
2019-08-14 10:16:00.255179 231292 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/T_Quadrant I_S_H_2019Q1.csv
2019-08-14 10:16:00.901441 231292 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/T_Quadrant I_S_L_2019Q1.csv
2019-08-14 10:16:01.705763 231291 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/T_Quadrant I_T_H_2019Q1.csv
2019-08-14 10:16:02.337720 231290 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/T_Quadrant I_T_M_2019Q1.csv
2019-08-14 10:16:02.499530 231290 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/C_Quadrant III_P_H_2

2019-08-14 10:19:44.722613 231253 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/T_Quadrant IV_S_M_2019Q1.csv
2019-08-14 10:19:45.445120 231253 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/T_Quadrant IV_T_H_2019Q1.csv
2019-08-14 10:19:45.669824 231253 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/T_Quadrant IV_T_L_2019Q1.csv
2019-08-14 10:19:46.224237 231252 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/T_Quadrant IV_T_M_2019Q1.csv
2019-08-14 10:22:49.043554 231241 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/T_Quadrant I_P_H_2019Q1.csv
2019-08-14 10:23:42.649131 231233 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/T_Quadrant I_P_M

2019-08-14 10:23:55.065154 231227 /home/jian/celery/Append_to_celery_LiveRamp/monthly_update/output_2019-06-29/NewReward_2019-06-29_T.csv
2019-08-14 10:23:56.164772 231227 /home/jian/celery/Append_to_celery_LiveRamp/monthly_update/output_2019-07-13/NewReward_2019-07-13_P.csv
2019-08-14 10:23:56.698792 231227 /home/jian/celery/Append_to_celery_LiveRamp/monthly_update/output_2019-07-13/NewReward_2019-07-13_S.csv
2019-08-14 10:23:56.988348 231227 /home/jian/celery/Append_to_celery_LiveRamp/monthly_update/output_2019-07-13/NewReward_2019-07-13_T.csv
2019-08-14 10:23:58.496234 231227 /home/jian/celery/Append_to_celery_LiveRamp/monthly_update/output_2019-07-27/NewReward_2019-07-27_P.csv
2019-08-14 10:23:58.971112 231227 /home/jian/celery/Append_to_celery_LiveRamp/monthly_update/output_2019-07-27/NewReward_2019-07-27_S.csv
2019-08-14 10:24:00.251106 231227 /home/jian/celery/Append_to_celery_LiveRamp/monthly_update/output_2019-07-27/NewReward_2019-07-27_T.csv


In [10]:
df_all_new_master = df_all_new_master.drop_duplicates('email_address_hash')
df_all_new_master = df_all_new_master.drop_duplicates('customer_id_hashed')
print(len(df_all_new_master.index))

231227


In [11]:
zipmap = pd.ExcelFile('/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx')
zipmap=zipmap.parse("unique_zips_full_footprint",dtype=str)
zipmap['zip_type']=zipmap['zip_type'].replace("zips_10","trans_S")

zipmap['zipcodegroup'] = zipmap['zip_type'].apply(lambda x: x.split("_")[1])
print(zipmap.shape)
print(zipmap['zip_cd'].nunique())

zipmap=zipmap.rename(columns={"zip_cd":"customer_zip_code"})

zipmap=zipmap[['customer_zip_code','zipcodegroup']]
zipmap.head(2)

(12980, 3)
12980


,customer_zip_code,zipcodegroup
0,75040,P
1,20743,P


In [12]:
df_all_new_master = pd.merge(df_all_new_master,zipmap,on ='customer_zip_code',how = 'left' )
print(df_all_new_master['zipcodegroup'].unique())

df_all_new_master['zipcodegroup'].fillna('T',inplace = True)
print(df_all_new_master['zipcodegroup'].unique())

last_saturday_str="_"+str(last_saturday).split("-")[1]+str(last_saturday).split("-")[2]+str(last_saturday).split("-")[0][2:]+"_"

for revenue_flag in ['P','S','T']:
    df = df_all_new_master[df_all_new_master['zipcodegroup']==revenue_flag]
    df = df[['customer_id_hashed','email_address_hash','customer_zip_code']]
    df['segment'] = 'NewReward' + last_saturday_str + revenue_flag
    print(revenue_flag,len(df.index))
    df.to_csv(write_folder+'NewReward_'+str(last_saturday)+'_'+revenue_flag+'.csv',index = False)
# To be uploaded to lr-big-lots /uploads/big_lots_onboarding

['P' 'S' nan]
['P' 'S' 'T']
P 101780
S 42346
T 87101


In [13]:
df_all_new_master.groupby(['zipcodegroup'])['email_address_hash'].count().to_frame().reset_index()

,zipcodegroup,email_address_hash
0,P,101780
1,S,42346
2,T,87101


In [14]:
df.head(2)

,customer_id_hashed,email_address_hash,customer_zip_code,segment
6,b7995c9dd0df9b8d8db45c9e507cac785ce9afe0f1d28b...,5119c565c237d8c6f8cc8fe88e41f4294573a472b06802...,NaN,NewReward_081019_T
10,b5a88b329a22968ff4222371b2f448afa37f549614d400...,82781cee82b5742a07bf351c0620e9e1d0c70bb438a152...,NaN,NewReward_081019_T


In [15]:
print(datetime.datetime.now())

2019-08-14 10:24:09.740950


In [16]:
import paramiko
import glob

host = "files.liveramp.com"
port = 22
password = "Biglots2018!"
username = "lr-big-lots"

transport = paramiko.Transport((host, port))

transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)

# local_path defined above before saving the local txt
local_list=glob.glob(write_folder+"*.csv")
local_list=[x for x in local_list if 'NewReward_'+str(last_saturday)+'_' in x]

for local_path in local_list:
    remote_path="/uploads/big_lots_onboarding/"+os.path.basename(local_path)
    sftp.put(local_path,remote_path)
sftp.close()
transport.close()